<a href="https://colab.research.google.com/github/IppoPowerOptimizer/Modelo_Hidrotermico_Uninodal/blob/main/Mod_Hidro_Uni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the pyomo module
from pyomo.environ import *
import pandas as pd

In [ ]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

# **Datos de entrada**

In [ ]:
data1 = {
    'c':    {'gh1': 0.8},
    'Pmin_h': {'gh1': 0},
    'Pmax_h': {'gh1': 450},
    'meta'  : {'gh1': 1.5}
}
data2 = {
    'ci':   {'gt1': 5},
    'Pmin_t': {'gt1': 0},
    'Pmax_t': {'gt1': 600}
}
data3 = {
    'T1': 8, 'T2': 10, 'T3': 6,
}
data4 = {
    'T1': 350, 'T2': 700, 'T3': 500
}



# **modelo**

In [ ]:
model = ConcreteModel()

## **conjunto**

In [ ]:
#  Sets
model.GH  = Set(initialize=['gh1'])
model.GT  = Set(initialize=['gt1'])
model.T = Set(initialize=data3.keys())

## **parámetros**

In [ ]:
model.c           = Param(model.GH, initialize=data1['c'])
model.Pmin_h      = Param(model.GH, initialize=data1['Pmin_h'])
model.Pmax_h      = Param(model.GH, initialize=data1['Pmax_h'])
model.meta        = Param(model.GH, initialize=data1['meta'])
model.ci          = Param(model.GT, initialize=data2['ci'])
model.Pmin_t      = Param(model.GT, initialize=data2['Pmin_t'])
model.Pmax_t      = Param(model.GT, initialize=data2['Pmax_t'])
model.c_rac       = Param(initialize=8000, within=NonNegativeReals)
model.Demand      = Param(model.T, initialize=data4)
model.period      = Param(model.T, initialize=data3)

## **variables**

In [ ]:
model.pt          = Var(model.GT, model.T, within=NonNegativeReals)
model.ph          = Var(model.GH, model.T, within=NonNegativeReals)
model.racion      = Var(model.T, within=NonNegativeReals)

### **limites de las variables**

In [ ]:
for gh in model.GH:
  for t in model.T:
    model.ph[gh, t].setlb(model.Pmin_h[gh])
    model.ph[gh, t].setub(model.Pmax_h[gh])

In [ ]:
for gt in model.GT:
  for t in model.T:
    model.pt[gt, t].setlb(model.Pmin_t[gt])
    model.pt[gt, t].setub(model.Pmax_t[gt])

## **Funcion objetivo**

In [ ]:
def funcion_objetivo(model):
    return sum(model.ci[gt] * model.pt[gt, t] * model.period[t] for gt in model.GT for t in model.T) \
         + sum(model.racion[t] * model.c_rac * model.period[t] for t in model.T)

model.objective = Objective(rule=funcion_objetivo, sense=minimize, doc='Define objective function')



## **restricciones**

In [ ]:
def BE_rule(model, t):
    return (sum(model.pt[gt, t] for gt in model.GT) \
         + sum(model.ph[gh, t] for gh in model.GH) \
         + model.racion[t]) * model.period[t] == model.Demand[t] * model.period[t]

model.BE = Constraint(model.T, rule=BE_rule)


In [ ]:
def BH_rule(model, gh):
    return sum(model.c[gh] * model.ph[gh, t] * model.period[t] * 3600 for t in model.T) <= model.meta[gh] * 10**6

model.BH = Constraint(model.GH, rule=BH_rule)

In [ ]:
model.BH.pprint()

BH : Size=1, Index=GH, Active=True
    Key : Lower : Body                                                                    : Upper     : Active
    gh1 :  -Inf : 23040.0*ph[gh1,T1] + 28800.0*ph[gh1,T2] + 17280.000000000004*ph[gh1,T3] : 1500000.0 :   True


In [ ]:
solver = SolverFactory("cbc")
results = solver.solve(model, tee=True).write()

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -printingOptions all -import /tmp/tmp43pi0sgk.pyomo.lp -stat=1 -solve -solu /tmp/tmp43pi0sgk.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 2 (-2) rows, 5 (-4) columns and 6 (-6) elements
Statistics for presolved model


Problem has 2 rows, 5 columns (4 with objective) and 6 elements
There are 4 singletons with objective 
Column breakdown:
1 of type 0.0->inf, 4 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
2 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
0 of type L other, 0 of type Range 0.0->1.0, 0 of type Range other, 
0 of type Free 
Presolve 2 (-2) rows, 5 (-4) columns and 6 (-6) elements
0  Obj 38517

In [ ]:
print('The Operation Cost:', "{:.2f}".format(value(model.objective)))

The Operation Cost: 3892333.36


In [ ]:
print("Constraints in the model:")
for constraint in model.component_objects(Constraint, active=True):
  print(f"Constraint: {constraint.name}")
  constraint.pprint()
  print()

Constraints in the model:
Constraint: BE
BE : Size=3, Index=T, Active=True
    Key : Lower  : Body                                      : Upper  : Active
     T1 : 2800.0 :  (pt[gt1,T1] + ph[gh1,T1] + racion[T1])*8 : 2800.0 :   True
     T2 : 7000.0 : (pt[gt1,T2] + ph[gh1,T2] + racion[T2])*10 : 7000.0 :   True
     T3 : 3000.0 :  (pt[gt1,T3] + ph[gh1,T3] + racion[T3])*6 : 3000.0 :   True

Constraint: BH
BH : Size=1, Index=GH, Active=True
    Key : Lower : Body                                                                    : Upper     : Active
    gh1 :  -Inf : 23040.0*ph[gh1,T1] + 28800.0*ph[gh1,T2] + 17280.000000000004*ph[gh1,T3] : 1500000.0 :   True

